# Parte 02: Feature Pipeline


## 🗒️ Este notebook é dividido em duas sessões:
1. Análise de Dados.
2. Inserção de novos dados ao Feature Store.

## 📝 Importando os Pacotes

In [ ]:
# Importando os Pacotes
from datetime import timedelta, datetime
import pandas as pd
import os

from features import citibike, meteorological_measurements

# Mutando os warnings
import warnings
warnings.filterwarnings("ignore")

---

## Conectando ao Hopsworks Feature Store

In [ ]:
# Importando os pacotes
import hopsworks

# Login na Hopsworks
project = hopsworks.login()

# Carregando o Feature Store
fs = project.get_feature_store()

In [ ]:
# Recuperar Feature Group do citibike_usage e meteorological_measurements
citibike_usage_fg = fs.get_or_create_feature_group(
    name="citibike_usage",
    version=1,
)

meteorological_measurements_fg = fs.get_or_create_feature_group(
    name="meteorological_measurements",
    version=1,
)


### 📅 Definindo a Data Mais Recente


In [ ]:
# Definindo a última data de 'citibike_usage_fg' no Feature Group
last_date = citibike.get_last_date_in_fg(citibike_usage_fg)
last_date

In [ ]:
# Definindo o dia seguinte à 'last_date'
next_date = citibike.get_next_date(last_date).split("-")
next_date

In [ ]:
# Extrair o ano e o mês-alvo da próxima data
target_year, target_month = int(next_date[0]), int(next_date[1])

print(f"Vamos baixar os dados do Citibike para {target_month}/{target_year}")


---

## 🪄 Analisando Novos Dados

### 🚲 Informações Sobre o Uso do Citibike

In [ ]:
# Baixar dados do Citibike para o mês e ano especificados
df_raw_batch = citibike.get_citibike_data(
    f"{target_month}/{target_year}", 
    f"{target_month}/{target_year}",
)
df_raw_batch.head(3)


In [ ]:
# Feature Engineering do Citibike para o lote de dados baixado
df_enhanced_batch = citibike.engineer_citibike_features(
    df_raw_batch,
)


In [ ]:
# Drop rows with missing values in the enhanced batch DataFrame
df_enhanced_batch = df_enhanced_batch.dropna()

# Convert 'station_id' to string type for categorical representation
df_enhanced_batch.station_id = df_enhanced_batch.station_id.astype(str)

# Display the last three rows of the enhanced batch DataFrame
df_enhanced_batch.tail(3)

### 🌤 Medidas Meteorológicas do VisualCrossing<

Você analisará dados meteorológicos, portanto, deve obter uma chave de API do [VisualCrossing](https://www.visualcrossing.com/). Você pode usar [este link](https://www.visualcrossing.com/weather-api).

**Não se esqueça de criar um arquivo de configuração `.env` dentro deste diretório, onde todas as variáveis de ambiente necessárias serão armazenadas:**

`WEATHER_API_KEY = "SUA_CHAVE_DE_API"`

> Se você fizer isso depois de executar este notebook, reinicie o Kernel do Python (porque `functions.py` não terá essas variáveis em seu namespace).

![](images/api_keys_env_file.png)

In [ ]:
# Converter a coluna 'date' para o tipo string
df_enhanced_batch.date = df_enhanced_batch.date.astype(str)

# Encontrar as datas mínima e máxima na coluna 'date' do DataFrame de lote aprimorado
start_date, end_date = df_enhanced_batch.date.min(), df_enhanced_batch.date.max()

In [ ]:
# Obter dados meteorológicos para a cidade de Nova York dentro do intervalo de datas do DataFrame de lote aprimorado
df_weather_batch = meteorological_measurements.get_weather_data(
    city="nyc", 
    start_date=start_date, 
    end_date=end_date,
)
df_weather_batch.tail(3)


In [ ]:
# Corrigir tipos de dados para colunas especificadas no DataFrame de lote de clima
for column in ["snowdepth", "snow"]:
    df_weather_batch[column] = df_weather_batch[column].astype("double")


In [ ]:
# Criação das colunas Unix
df_enhanced_batch["timestamp"] = df_enhanced_batch["date"].apply(
    meteorological_measurements.convert_date_to_unix
)
df_weather_batch["timestamp"] = df_weather_batch["date"].apply(
    meteorological_measurements.convert_date_to_unix
)


---

## ⬆️ Enviando Novos Dados Para o Feature Store

In [ ]:
# Inserir novos dados
citibike_usage_fg.insert(df_enhanced_batch)

In [ ]:
# Inserir novos dados
meteorological_measurements_fg.insert(
    df_weather_batch,
    write_options={"wait_for_job": True},
)


---

## ⏭️ **Próximo:** Parte 03: Pipeline de Treinamento

No próximo notebook, você criará um Feature View, conjunto de dados de treinamento, treinará um modelo e o registrará no Registro de Modelos do Hopsworks.